In [2]:
import pandas as pd

In [3]:
metag = pd.read_csv("ENA_tables/PRJEB4352_metaG_wenv_PE.txt", sep = '\t')
metat = pd.read_csv("ENA_tables/PRJEB6609_metaT_wenv_PE.txt", sep = '\t')

metag['size_frac']= metag['Fraction lower [µm]'].map(str) + '-' + metag['Fraction upper [µm]'].map(str)
metat['size_frac']= metat['Fraction lower [µm]'].map(str) + '-' + metat['Fraction upper [µm]'].map(str)

# Tidy Size fraction data splitting 

In [31]:
# Original data -- group by size fractions in 4 distinct groups
pd.DataFrame(metag.groupby('size_frac').count()['study_accession'])

,study_accession
size_frac,
0.8-5.00,339
180-2000.00,179
20-180.00,167
5-20.00,136


In [32]:
#drop 0.22 sample
metag = metag[metag.size_frac != '0.22-3.00']
metag = metag.replace('0.8->0.80', '0.8-5.00')
metag = metag.replace('0.8-20.00', '0.8-5.00')
metag = metag.replace('0.8-3.00', '0.8-5.00')
metag = metag.replace('3.0->3.00', '0.8-5.00')
metag = metag.replace('3.0->5.00', '0.8-5.00')
metag = metag.replace('180.0-2000.00', '180-2000.00')
metag = metag.replace('20.0-180.00', '20-180.00')
metag = metag.replace('5.0-20.00', '5-20.00')

#Check final groupings
pd.DataFrame(metag.groupby('size_frac').count()['study_accession'])

,study_accession
size_frac,
0.8-5.00,339
180-2000.00,179
20-180.00,167
5-20.00,136


In [33]:
#drop 0.22 sample
metat = metat.replace('0.8->0.80', '0.8-5.00')
metat = metat.replace('0.8-20.00', '0.8-5.00')
metat = metat.replace('0.8-3.00', '0.8-5.00')
metat = metat.replace('3.0->3.00', '0.8-5.00')
metat = metat.replace('3.0->5.00', '0.8-5.00')
metat = metat.replace('180.0-2000.00', '180-2000.00')
metat = metat.replace('20.0-180.00', '20-180.00')
metat = metat.replace('5.0-20.00', '5-20.00')


pd.DataFrame(metat.groupby('size_frac').count()['study_accession'])

,study_accession
size_frac,
0.8-5.00,316
180-2000.00,183
20-180.00,178
5-20.00,164


# Check Ocean Region and Depth and clean

In [34]:
metag['OS'] = metag['OS region'].str.split(' ').str[0].str.strip('[').str.strip(']')
metag['EF'] = metag['Env feature'].str.split(' ').str[0].str.strip('[').str.strip(']')
metag['BG'] = metag['BG province'].str.split(' ').str[0].str.strip('[').str.strip(']')


In [35]:
metat['OS'] = metat['OS region'].str.split(' ').str[0].str.strip('[').str.strip(']')
metat['EF'] = metat['Env feature'].str.split(' ').str[0].str.strip('[').str.strip(']')
metat['BG'] = metat['BG province'].str.split(' ').str[0].str.strip('[').str.strip(']')


In [36]:
subset = ['study_accession', 'run_accession', 'Station', 'size_frac', 'Env feature', 
        'Sample material', 'Depth, nominal', 'OS region', 'BG province',
        'Latitude', 'Longitude', 'OS', 'EF', 'BG']

metag_subset = metag[subset]
metat_subset = metat[subset]

# Function to split into groups

In [37]:
def split_assembly_groups(df, OS_list):
    df['ERR_list']= df.groupby(['OS', 'EF', 'size_frac'])['run_accession'].transform(lambda x: ', '.join(x))
    df_combo = df[['OS', 'EF', 'size_frac','ERR_list']].drop_duplicates()
    df = df.drop('ERR_list', axis =1)
    for O in OS_list:
        df_combo = df_combo[df_combo.OS != O]
        tmp = df[df.OS == O]
        tmp['ERR_list']=tmp.groupby(['OS','BG', 'EF', 'size_frac'])['run_accession'].transform(lambda x: ', '.join(x))
        tmp_combo =tmp[['OS', 'BG', 'EF', 'size_frac','ERR_list']].drop_duplicates()
    final_combo = pd.concat([df_combo, tmp_combo])  
    final_combo = final_combo.fillna('all')
    final_combo['Assembly_group'] = final_combo.OS + '-' + final_combo.BG + '-' + final_combo.EF + '-' +final_combo.size_frac
    final_combo['Sub_region'] = final_combo.OS + '-' + final_combo.BG
    final_combo['Depth_sizefrac'] = final_combo.EF + '-' +final_combo.size_frac
    final_combo['ERR_count']= final_combo['ERR_list'].str.split(', ').str.len()
    final_combo = final_combo[['Sub_region', 'Depth_sizefrac', 'ERR_count', 'ERR_list', 'Assembly_group']]
    final_combo = final_combo.set_index('Sub_region')
    return(final_combo)

In [38]:
OS_list = ['SPO']
metag_out = split_assembly_groups(metag, OS_list)
metat_out = split_assembly_groups(metat, OS_list)

metag_out.to_csv('SampleList_ForAssembly_metaG_python.txt', sep = '\t')
metat_out.to_csv('SampleList_ForAssembly_metaT_python.txt', sep = '\t')

/vortexfs1/home/halexander/.conda/envs/sourmash-mag-euk/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/vortexfs1/home/halexander/.conda/envs/sourmash-mag-euk/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [39]:
metag_out.sort_values(['ERR_count'])

,Depth_sizefrac,ERR_count,ERR_list,Assembly_group
Sub_region,,,,
SPO-CHIL,MES-0.8-5.00,1,ERR868377,SPO-CHIL-MES-0.8-5.00
SPO-PEOD,MES-0.8-5.00,1,ERR868362,SPO-PEOD-MES-0.8-5.00
SPO-CHIL,DCM-20-180.00,1,ERR1726830,SPO-CHIL-DCM-20-180.00
RS-all,DCM-5-20.00,1,ERR1726910,RS-all-DCM-5-20.00
SO-all,DCM-20-180.00,1,ERR599288,SO-all-DCM-20-180.00
SPO-CHIL,DCM-5-20.00,2,"ERR1700901, ERR1726828",SPO-CHIL-DCM-5-20.00
NAO-all,MIX-5-20.00,2,"ERR1726545, ERR1726756",NAO-all-MIX-5-20.00
SPO-PEOD,DCM-180-2000.00,2,"ERR1700911, ERR1726962",SPO-PEOD-DCM-180-2000.00
NAO-all,MIX-180-2000.00,2,"ERR1726532, ERR1726941",NAO-all-MIX-180-2000.00


In [40]:
sara = pd.read_csv('SampleList_ForAssembly.txt', sep = '\t', index_col = 'Sub_region')


In [21]:
set(metag_out.Depth_sizefrac.unique()).difference(set(sara.Depth_sizefrac.unique()))

{'DCM-180.0-2000.00',
 'DCM-20.0-180.00',
 'DCM-5.0-20.00',
 'MIX-180.0-2000.00',
 'MIX-20.0-180.00',
 'MIX-5.0-20.00',
 'SRF-180.0-2000.00',
 'SRF-20.0-180.00',
 'SRF-5.0-20.00'}

In [24]:
set(metag.size_frac)

{'0.8-5.00', '180.0-2000.00', '20.0-180.00', '5.0-20.00'}

In [25]:
sara

,Depth_sizefrac,ERR_count,ERR_list,Assembly_group
Sub_region,,,,
IO-all,DCM-0.8-5.00,13,"ERR1726633, ERR1726897, ERR599323, ERR599324, ...",IO-all-DCM-0.8-5.00
IO-all,DCM-180-2000.00,12,"ERR1726526, ERR1726686, ERR1726696, ERR1726746...",IO-all-DCM-180-2000.00
IO-all,DCM-20-180.00,10,"ERR1726554, ERR1726590, ERR1726671, ERR1726736...",IO-all-DCM-20-180.00
IO-all,DCM-5-20.00,6,"ERR1700904, ERR1700912, ERR1726665, ERR599205,...",IO-all-DCM-5-20.00
IO-all,FSW-0.8-5.00,2,"ERR868368, ERR868484",IO-all-FSW-0.8-5.00
IO-all,MES-0.8-5.00,8,"ERR599212, ERR599225, ERR599268, ERR868353, ER...",IO-all-MES-0.8-5.00
IO-all,SRF-0.8-5.00,20,"ERR1726549, ERR1740328, ERR599210, ERR599266, ...",IO-all-SRF-0.8-5.00
IO-all,SRF-180-2000.00,21,"ERR1700889, ERR1700906, ERR1700907, ERR1700909...",IO-all-SRF-180-2000.00
IO-all,SRF-20-180.00,17,"ERR1700914, ERR1726587, ERR1726599, ERR1726611...",IO-all-SRF-20-180.00


94